In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import zscore, pearsonr, kendalltau,  entropy, spearmanr, linregress, rankdata, ttest_rel, ttest_1samp, ttest_ind
from statsmodels.stats import multitest
import scipy.linalg as la
import itertools
import random
import math
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from statsmodels.stats import multitest
import nibabel as nib


# %% 불용어 리스트 불러오기
# input은 커스텀 불용어
def load_stopword(input_list=None):
    stopwords_f = open('stopwords.txt','rt', encoding='UTF8')
    stopwords_list = stopwords_f.readlines()
    stopwords_f.close()
    stopwords = []
    for words in stopwords_list:
        words = words.replace("\n", '')
        stopwords.append(words.split('/'))
    if input_list:
        for words in input_list: stopwords.append([words[0], words[1]])
    return(stopwords)


    
# %% 구어 태깅
#  구어 태깅
def etri_spokentagger(input, stopwords=False):
    """_ETRI 구어 태깅_

    Args:
        input (_str_): 텍스트
        stopwords (optional): 불용어 제외 여부
            Defaults to True, 기본 불용어 (load_stopword)
            False: 하지 않음
            list: [["단어", "품사]] 형태의 커스텀 불용어
    """
    import urllib3
    import json
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
    accessKey = "9d3f4f60-ac44-43c9-9ae4-a3df0ee86686"  # 개인 accessKey
    analysisCode = "morp"
    requestJson = {
    "argument": {
        "text": input,
        "analysis_code": analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    results = response.data.decode('utf-8')
    results = json.loads(results)['return_object']['sentence']
    tagged_results = []
    # 불용어 제거
    if stopwords:
        stopwords_list = load_stopword()
        stopword_tag = []
        for word in stopwords_list:
            if not word[0]: stopword_tag.append(word[1])
    
    # 결과값 저장
    for sent_result in results:
        sentence = []
        for word in sent_result['morp']:
            if stopwords:
                if word['type'] in stopword_tag: pass
                elif [word['lemma'], word['type']] in stopwords_list: pass
                else: sentence.append([word['lemma'], word['type']])
            else: sentence.append([word['lemma'], word['type']])
        tagged_results.extend(sentence)  # rather use extend  
    return(tagged_results)

def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

def extract_recalledidx(group):
    keys = [f'sub-0{group}01',f'sub-0{group}02',f'sub-0{group}03']

    data_transcript = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
    sharedeb = np.array(data_transcript['sharedeb'])
    data_recs = np.array(data_transcript[keys])

    recalled_evidx=[]
    recalled_evs = []
    for i in range(len(sharedeb)):
        count=0
        for j in range(3):
            if (type(data_recs[i,j])==str or np.isnan(data_recs[i,j])==False) and (data_recs[i,j]!=0):
                count+=1
        if count==3:
            if sharedeb[i] not in recalled_evidx:
                recalled_evidx.append(sharedeb[i])
                recalled_evs.append(data_recs[i,:])
    return recalled_evs

def extract_recalledidx_all(group):
    keys = [f'sub-0{group}01',f'sub-0{group}02',f'sub-0{group}03']

    data_transcript = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
    sharedeb = np.array(data_transcript['sharedeb'])
    data_recs = np.array(data_transcript[keys])
    recalled_evidx_all_oh = np.zeros((sharedeb[-1],3))
    recalled_evs_all = np.zeros((sharedeb[-1],3), dtype='object')
    for i in range(len(sharedeb)):
        for j in range(3):
            if (type(data_recs[i,j])==str or np.isnan(data_recs[i,j])==False) and (data_recs[i,j]!=0):
                recalled_evidx_all_oh[int(sharedeb[i])-1,j] = 1
                recalled_evs_all[int(sharedeb[i])-1,j] = str(data_recs[i,j])
    return recalled_evidx_all_oh, recalled_evs_all

def extract_recalledidx_all_sharedeb(group):
    keys = [f'sub-0{group}01',f'sub-0{group}02',f'sub-0{group}03']

    data_transcript = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
    sharedeb = np.array(data_transcript['sharedeb'])
    data_recs = np.array(data_transcript[keys])
    recalled_evidx_all_oh = np.zeros((sharedeb[-1],3))
    recalled_evs_all = np.zeros((sharedeb[-1],3), dtype='object')
    for i in range(len(sharedeb)):
        for j in range(3):
            if (type(data_recs[i,j])==str or np.isnan(data_recs[i,j])==False) and (data_recs[i,j]!=0):
                recalled_evidx_all_oh[int(sharedeb[i])-1,j] = 1
                recalled_evs_all[int(sharedeb[i])-1,j] = str(data_recs[i,j])
    recalled_evs_all = np.column_stack((np.arange(1,max(sharedeb)+1),recalled_evs_all))
    return recalled_evidx_all_oh, recalled_evs_all

def get_boundary(group,sub):
    groupsub = group+sub
    word_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_AUDIO/derivatives/group-{group}/sub-{groupsub}_run-1_day2_words.xlsx')
    posthoc_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV/group-{group}/sub-{groupsub}_day2_posthoc.xlsx')
    # word_data.keys() = start, end, word
    # posthoc_data.keys() = sentence, segmentation, description, tag
    words = word_data['word']
    start = word_data['start']
    end = word_data['end']
    sentstartend = []
    eventstartend = []

    segcol = posthoc_data['segmentation']

    # 아래가 concat의 원리
    tmpstart = start[0]
    for i in range (len(words)-1):
        if '.' in str(words[i]) or ',' in str(words[i]) or '었고' in str(words[i]) or '것 같고' in str(words[i]) or '지고' in str(words[i]) or '는데' in str(words[i]) or '근데' in str(words[i]) or '그리고' in str(words[i]) or '그러고' in str(words[i]) or '됐고' in str(words[i]) or '했고' in str(words[i]): 
            tmpend = end[i]
            sentstartend.append([tmpstart,tmpend])
            tmpstart = start[i+1]
    tmpend = end[len(words)-1]
    sentstartend.append([tmpstart,tmpend])

    tmpstart = sentstartend[0][0]
    boundary = []
    for i in range (len(segcol)-1):
        if segcol[i] != segcol[i+1]:
            boundary.append(sentstartend[i][1])            
            eventstartend.append([tmpstart,sentstartend[i][1]])
            tmpstart = sentstartend[i+1][0]
    tmpend = sentstartend[-1][1]
    eventstartend.append([tmpstart,tmpend])    
    
    return eventstartend, boundary

def get_boundary2(group,sub,run):
    groupsub = '0'+str(group)+'0'+str(sub)
    word_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_AUDIO/derivatives/group-0{group}/sub-{groupsub}_run-{run}_day2_words.xlsx')
    posthoc_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{group}/sub-{groupsub}_day2_posthoc.xlsx')
    # word_data.keys() = start, end, word
    # posthoc_data.keys() = sentence, segmentation, description, tag
    words = word_data['word']
    start = word_data['start']
    end = word_data['end']
    sentstartend = []
    eventstartend = []
    endidx = [0]
    for t in range (len(posthoc_data['Unnamed: 0'])-1):
        if posthoc_data['Unnamed: 0'][t]+1 != posthoc_data['Unnamed: 0'][t+1]:
            endidx.append(t)
    endidx.append(len(posthoc_data['Unnamed: 0'])-1)
    if len(endidx)>2:
        segcol = np.array(posthoc_data['segmentation'][endidx[run-1]+1:endidx[run]+1])
    else:
        segcol = np.array(posthoc_data['segmentation'])

    tmpstart = start[0]
    for i in range (len(words)-1):
        if '.' in str(words[i]) or ',' in str(words[i]) or '었고' in str(words[i]) or '것 같고' in str(words[i]) or '지고' in str(words[i]) or '는데' in str(words[i]) or '근데' in str(words[i]) or '그리고' in str(words[i]) or '그러고' in str(words[i]) or '됐고' in str(words[i]) or '했고' in str(words[i]): 
            tmpend = end[i]
            sentstartend.append([tmpstart,tmpend])
            tmpstart = start[i+1]
    tmpend = end[len(words)-1]
    sentstartend.append([tmpstart,tmpend])

    # working for subs with only 1 run
    tmpstart = sentstartend[0][0]
    for i in range (len(segcol)-1):
        if segcol[i] != segcol[i+1]:     
            tmpend = sentstartend[i][1]     
            eventstartend.append([tmpstart,tmpend])
            tmpstart = sentstartend[i+1][0]
    tmpend = sentstartend[len(segcol)-1][1]
    eventstartend.append([tmpstart,tmpend]) 
    
    return eventstartend

def r2z(r):
    return 0.5 * (np.log(1 + r) - np.log(1 - r))

In [7]:
DATADIR = f'/home/jiunchoi/OFD/OFD_BHV_clean'
groups = [2,3,4,5]
group_clean_txts = np.load('output/group_clean_txts.npy',allow_pickle=True)

In [8]:
groups = [2,3,4,5]
tokenbags = np.zeros((len(groups),3),dtype='object') #group,sub,s/us

for g,group in enumerate(groups):
    recalled_evs = np.array(extract_recalledidx(group))
    for sub in [1,2,3]:
        data_transcript = pd.read_excel(f'{DATADIR}/group-0{str(group)}/sub-0{str(group)}0{str(sub)}_day2_posthoc.xlsx')
        recall_segmentation = np.array(data_transcript['segmentation'])
        sub_ev = np.zeros(int(recall_segmentation[-1]),dtype='object')
        for e in range(1,int(recall_segmentation[-1])+1):
            sentfinderidx = np.where(recall_segmentation==e)[0]
            sentlevel = []
            for eachsent in group_clean_txts[group-2,sub-1][sentfinderidx]:
                sentlevel.extend(eachsent)
            sub_ev[e-1] = sentlevel
        tokenbags[g,sub-1] = sub_ev


In [9]:
dfBAG = []
posh = []
for g,group in enumerate([2,3,4,5]):
    BOW_of_group = []
    for sub in [1,2,3]:
        BOW_of_group.extend(itertools.chain.from_iterable(list(tokenbags[g,sub-1])))
        A = list(set(tuple(i) for i in BOW_of_group))

    evBOW_of_group = []
    for sub in [1,2,3]:
        subn_evtotal_oh = np.zeros((len(A),len(tokenbags[g,sub-1])))
        for ev in range(len(tokenbags[g,sub-1])):
            subn_evn = set(tuple(i) for i in tokenbags[g,sub-1][ev])

            ev_label = []
            for idx,answer in enumerate(A):
                for pair in subn_evn:
                    if answer==pair:
                        ev_label.append(idx)
            for tokenidx in ev_label:
                subn_evtotal_oh[tokenidx,ev] = 1
        evBOW_of_group.append(subn_evtotal_oh)
    print(group, evBOW_of_group[0].shape, evBOW_of_group[1].shape, evBOW_of_group[2].shape, 'checkpoint2')

    # shared ev면 그것끼리 mean을 해 (이걸 len(recalled_evs) 길이의 3차원으로 만드셈)
    recalled_evs = np.array(extract_recalledidx(group))
    event_word_representation = np.zeros_like(recalled_evs, dtype='object')
    for e,ev in enumerate(recalled_evs):
        tmp_ev_word_representation = []
        for sub in [1,2,3]:
            event_no = str(ev[sub-1]).split(',') #list of str
            for en in event_no:
                tmp_ev_word_representation.append(evBOW_of_group[sub-1][:,int(float(en)-1)]) #이게 (1465,) vector
            subtmp_ev_word_representation = np.array(tmp_ev_word_representation)
            sub_ev_representation = np.mean(subtmp_ev_word_representation, axis=0)
            event_word_representation[e,sub-1] = sub_ev_representation
    mean_sharedev_word_representation = np.mean(event_word_representation,axis=1)
    print(group, recalled_evs.shape, mean_sharedev_word_representation.shape, 'checkpoint3')

    # 위의 sharedev에서 가장 비슷한 (본인 구하는데 들어간거 제외) - 매트릭스유사도비교 (euclidean distance/cossim)
    recalled_evidx_all_oh, recalled_evs_all = extract_recalledidx_all(group)
    recalled_ppls_word_representation = np.zeros_like(recalled_evidx_all_oh, dtype='object')

    for i,sharedev in enumerate(recalled_evs_all):
        tmp_shared_word_representation = []
        recalled_ppl = np.where(recalled_evidx_all_oh[i]==1)[0]
        # print(f'{i}번째 이벤트를 {recalled_ppl} 참가자가 리콜하였습니다')
        if len(recalled_ppl) > 0:
            for pplidx in recalled_ppl:
                event_no = str(sharedev[pplidx]).split(',') #list of str
                for en in event_no:
                    tmp_shared_word_representation.append(evBOW_of_group[pplidx][:,int(float(en)-1)])
                subtmp_ev_word_representation = np.array(tmp_shared_word_representation)
                sub_ev_representation = np.mean(subtmp_ev_word_representation, axis=0)
                recalled_ppls_word_representation[i,pplidx] = sub_ev_representation
    print(group, 'checkpoint4')

    # 이제 한줄짜리 만들거임. 셋 다 리콜한건 nan처리, 셋다리콜하지않은것도 nan처리
    mean_recalled_ppls_word_representation = np.zeros(len(recalled_ppls_word_representation),dtype='object')
    for ev,rpwr_row in enumerate(recalled_ppls_word_representation):
        recalled_ppl = np.where(recalled_evidx_all_oh[ev]==1)[0]
        if len(recalled_ppl) == 3:
            mean_recalled_ppls_word_representation[ev] = 'nan'
        elif len(recalled_ppl) == 0:
            mean_recalled_ppls_word_representation[ev] = 'nan'
        else:
            mean_recalled_ppls_word_representation[ev] = np.mean(rpwr_row[recalled_evidx_all_oh[ev]==1])
    print(group, 'checkpoint5')

    # 이제 셋다리콜한것과 원핫벡터가 제일 비슷한 1,2명 shared event를 찾자! <<근데 계속 나오는 이벤트만 나오는게 좀 걸림
    pair_of_sharedev = np.zeros((len(mean_sharedev_word_representation),2), dtype='object')
    for m,mswp in enumerate(mean_sharedev_word_representation):
        tmp_cossim = np.zeros(len(mean_recalled_ppls_word_representation))
        for mr, mrpwr in enumerate(mean_recalled_ppls_word_representation):
            if type(mswp)==np.ndarray and mrpwr!='nan':
                tmp_cossim[mr] = cos_similarity(mswp,mrpwr)

        candid = np.where(tmp_cossim == np.max(tmp_cossim))[0]
        ball = 1
        while ball==1:
            for r, reac in enumerate(recalled_evs_all[candid]):
                for s in range(3):
                    for cand in candid:
                        testee = str(reac[s]).split(',')
                        for tt in testee:
                            for res in (recalled_evs):
                                ev = str(res[s]).split(',')
                                for e in ev:
                                    if int(float(tt)) == int(float(e)):
                                        tmp_cossim[cand] = 0
                                    else:
                                        candid = np.where(tmp_cossim == np.max(tmp_cossim))[0]
                                        ball = 0
        pair_of_sharedev[m,0] = candid
        pair_of_sharedev[m,1] = tmp_cossim[candid]
    posh.append(pair_of_sharedev[:,0])
    print(group, 'checkpoint6')

    # 그리고 null distri도 만드셈 부트스트랩을 하든지
    idxlist = []
    for idx,mrpwr in enumerate(mean_recalled_ppls_word_representation):
        if mrpwr=='nan':
            pass
        else:
            idxlist.append(idx)

    n_permutation = 1000
    permuted_stats = np.zeros((n_permutation)) #save euclidean distance 
    for i in range(n_permutation):
        for m,mswr in enumerate(mean_sharedev_word_representation):
            for j in pair_of_sharedev[m,0]:
                try:
                    idxlist.remove(j)
                except:
                    pass
        nonpair_idx = random.choice(idxlist)
        permuted_stats[i] = cos_similarity(mswr,mean_recalled_ppls_word_representation[nonpair_idx])
    print(group, 'checkpoint7')

    # 위의 짓을 겁나많이해서 tmp_distance의 가능한 분포가 바이올린플롯으로 있고
    peridx = np.zeros_like(permuted_stats)
    pairidx = np.ones_like(pair_of_sharedev[:,1])
    a = np.vstack((peridx,permuted_stats))
    try:
        b = np.vstack((pairidx, pair_of_sharedev[:,1].astype('float')))
    except:
            pair_of_sharedev_dist = np.zeros(len(pair_of_sharedev[:,1]))
            for p,pose in enumerate(pair_of_sharedev[:,1]):
                if len(pose)>1:
                    pair_of_sharedev_dist[p] = pose[0]
                else:
                    pair_of_sharedev_dist[p] = pose
            b = np.vstack((pairidx, pair_of_sharedev_dist))

    c = np.hstack((a,b)).T
    df = pd.DataFrame(c, columns=['what','distance'],dtype='float')
    dfBAG.append(df)
    print(group, 'checkpoint8')
    # time.sleep(1)


2 (1484, 69) (1484, 27) (1484, 27) checkpoint2
2 (31, 3) (31,) checkpoint3
2 checkpoint4
2 checkpoint5


/tmp/ipykernel_523168/582355612.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if type(mswp)==np.ndarray and mrpwr!='nan':


2 checkpoint6


/tmp/ipykernel_523168/582355612.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mrpwr=='nan':


2 checkpoint7
2 checkpoint8
3 (952, 31) (952, 42) (952, 42) checkpoint2
3 (27, 3) (27,) checkpoint3
3 checkpoint4
3 checkpoint5


...

4 checkpoint7
4 checkpoint8
5 (773, 26) (773, 20) (773, 26) checkpoint2
5 (22, 3) (22,) checkpoint3
5 checkpoint4
5 checkpoint5


/tmp/ipykernel_523168/582355612.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if type(mswp)==np.ndarray and mrpwr!='nan':


5 checkpoint6
5 checkpoint7


/tmp/ipykernel_523168/582355612.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mrpwr=='nan':


5 checkpoint8


In [10]:
allcossim = []
dfBAG = []
posh = []

for g,group in enumerate([2,3,4,5]):
    BOW_of_group = []
    for sub in [1,2,3]:
        BOW_of_group.extend(itertools.chain.from_iterable(list(tokenbags[g,sub-1])))
        A = list(set(tuple(i) for i in BOW_of_group))

    evBOW_of_group = []
    for sub in [1,2,3]:
        subn_evtotal_oh = np.zeros((len(A),len(tokenbags[g,sub-1])))
        for ev in range(len(tokenbags[g,sub-1])):
            subn_evn = set(tuple(i) for i in tokenbags[g,sub-1][ev])

            ev_label = []
            for idx,answer in enumerate(A):
                for pair in subn_evn:
                    if answer==pair:
                        ev_label.append(idx)
            for tokenidx in ev_label:
                subn_evtotal_oh[tokenidx,ev] = 1
        evBOW_of_group.append(subn_evtotal_oh)
    # print(group, evBOW_of_group[0].shape, evBOW_of_group[1].shape, evBOW_of_group[2].shape, 'checkpoint2')

    # shared ev면 그것끼리 mean을 해 (이걸 len(recalled_evs) 길이의 3차원으로 만드셈)
    recalled_evs = np.array(extract_recalledidx(group))
    event_word_representation = np.zeros_like(recalled_evs, dtype='object')
    for e,ev in enumerate(recalled_evs):
        tmp_ev_word_representation = []
        for sub in [1,2,3]:
            event_no = str(ev[sub-1]).split(',') #list of str
            for en in event_no:
                tmp_ev_word_representation.append(evBOW_of_group[sub-1][:,int(float(en)-1)]) #이게 (1465,) vector
            subtmp_ev_word_representation = np.array(tmp_ev_word_representation)
            sub_ev_representation = np.mean(subtmp_ev_word_representation, axis=0)
            event_word_representation[e,sub-1] = sub_ev_representation
    mean_sharedev_word_representation = np.mean(event_word_representation,axis=1)
    # print(group, recalled_evs.shape, mean_sharedev_word_representation.shape, 'checkpoint3')

    # 위의 sharedev에서 가장 비슷한 (본인 구하는데 들어간거 제외) - 매트릭스유사도비교 (euclidean distance/cossim)
    recalled_evidx_all_oh, recalled_evs_all = extract_recalledidx_all(group)
    recalled_ppls_word_representation = np.zeros_like(recalled_evidx_all_oh, dtype='object')

    for i,sharedev in enumerate(recalled_evs_all):
        tmp_shared_word_representation = []
        recalled_ppl = np.where(recalled_evidx_all_oh[i]==1)[0]
        # print(f'{i}번째 이벤트를 {recalled_ppl} 참가자가 리콜하였습니다')
        if len(recalled_ppl) > 0:
            for pplidx in recalled_ppl:
                event_no = str(sharedev[pplidx]).split(',') #list of str
                for en in event_no:
                    tmp_shared_word_representation.append(evBOW_of_group[pplidx][:,int(float(en)-1)])
                subtmp_ev_word_representation = np.array(tmp_shared_word_representation)
                sub_ev_representation = np.mean(subtmp_ev_word_representation, axis=0)
                recalled_ppls_word_representation[i,pplidx] = sub_ev_representation
    # print(group, 'checkpoint4')

    # 이제 한줄짜리 만들거임. 셋 다 리콜한건 nan처리, 셋다리콜하지않은것도 nan처리
    mean_recalled_ppls_word_representation = np.zeros(len(recalled_ppls_word_representation),dtype='object')
    for ev,rpwr_row in enumerate(recalled_ppls_word_representation):
        recalled_ppl = np.where(recalled_evidx_all_oh[ev]==1)[0]
        if len(recalled_ppl) == 3:
            mean_recalled_ppls_word_representation[ev] = 'nan'
        elif len(recalled_ppl) == 0:
            mean_recalled_ppls_word_representation[ev] = 'nan'
        else:
            mean_recalled_ppls_word_representation[ev] = np.mean(rpwr_row[recalled_evidx_all_oh[ev]==1])
    # print(group, 'checkpoint5')

    # 이제 셋다리콜한것과 원핫벡터가 제일 비슷한 1,2명 shared event를 찾자! <<근데 계속 나오는 이벤트만 나오는게 좀 걸림
    pair_of_sharedev = np.zeros((len(mean_sharedev_word_representation),2), dtype='object')
    for m,mswp in enumerate(mean_sharedev_word_representation):
        tmp_cossim = np.zeros(len(mean_recalled_ppls_word_representation))
        for mr, mrpwr in enumerate(mean_recalled_ppls_word_representation):
            if type(mswp)==np.ndarray and mrpwr!='nan':
                tmp_cossim[mr] = cos_similarity(mswp,mrpwr)
                allcossim.append(tmp_cossim)

        candid = np.where(tmp_cossim == np.max(tmp_cossim))[0]
        ball = len(candid)
        while ball==len(candid):
            for cand in candid:
                if (np.sum(recalled_evidx_all_oh[cand])!=3) and (np.sum(recalled_evidx_all_oh[cand])!=0): #셋 다 말한 게 아니고 셋 다 안말한 것도 아니고
                    row = recalled_evs_all[cand]
                    for s,eachsub in enumerate(row):
                        esrec = str(eachsub).split(',') #recalled_evs_all에서 cand에 해당하는 줄
                        ms = str(mswp[s]).split(',') #얘랑 cossim을 계산했던 shared 한 줄
                        for esr in esrec: #참가자리콜 중 겹치는 event pattern이 없는지
                            for mss in ms:
                                if int(float(esr))!=int(float(mss)):
                                    ball-=1 # 저장을 해

        pair_of_sharedev[m,0] = candid
        pair_of_sharedev[m,1] = tmp_cossim[candid]
    posh.append(pair_of_sharedev[:,0])

    # 그리고 null distri도 만드셈 부트스트랩을 하든지
    idxlist = []
    for idx,mrpwr in enumerate(mean_recalled_ppls_word_representation):
        if mrpwr=='nan':
            pass
        else:
            idxlist.append(idx)

    n_permutation = 1000
    permuted_stats = np.zeros((n_permutation)) #save euclidean distance 
    for i in range(n_permutation):
        for m,mswr in enumerate(mean_sharedev_word_representation):
            for j in pair_of_sharedev[m,0]:
                try:
                    idxlist.remove(j)
                except:
                    pass
        nonpair_idx = random.choice(idxlist)
        permuted_stats[i] = cos_similarity(mswr,mean_recalled_ppls_word_representation[nonpair_idx])
    # print(group, 'checkpoint7')

    # 위의 짓을 겁나많이해서 tmp_distance의 가능한 분포가 바이올린플롯으로 있고
    peridx = np.zeros_like(permuted_stats)
    pairidx = np.ones_like(pair_of_sharedev[:,1])
    a = np.vstack((peridx,permuted_stats))
    try:
        b = np.vstack((pairidx, pair_of_sharedev[:,1].astype('float')))
    except:
            pair_of_sharedev_dist = np.zeros(len(pair_of_sharedev[:,1]))
            for p,pose in enumerate(pair_of_sharedev[:,1]):
                if len(pose)>1:
                    pair_of_sharedev_dist[p] = pose[0]
                else:
                    pair_of_sharedev_dist[p] = pose
            b = np.vstack((pairidx, pair_of_sharedev_dist))

    c = np.hstack((a,b)).T
    df = pd.DataFrame(c, columns=['what','distance'],dtype='float')
    dfBAG.append(df)
    # print(group, 'checkpoint8')

/tmp/ipykernel_523168/243270408.py:77: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if type(mswp)==np.ndarray and mrpwr!='nan':
/tmp/ipykernel_523168/243270408.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mrpwr=='nan':
/tmp/ipykernel_523168/243270408.py:77: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if type(mswp)==np.ndarray and mrpwr!='nan':
/tmp/ipykernel_523168/243270408.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mrpwr=='nan':
/tmp/ipykernel_523168/243270408.py:77: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if type(mswp)==np.ndarray and mrpwr!='nan

In [ ]:
# colors = sns.color_palette("Set2")
# plt.figure(figsize=(3,4),dpi=300)
# sns.violinplot(data= pd.concat(dfBAG), x='what',y='distance', pallette=colors)
# # plt.ylim(1,17)

# # 그럼에도 불구하고 내가 만들어준 페어는 토큰원핫벡터의 유사도가 sig하게 높다는 걸 보임
# df = pd.concat(dfBAG)
# t_stat, p_values = ttest_ind(df.loc[df['what']==0]['distance'], df.loc[df['what']==1]['distance'])
# print(i+2, t_stat, p_values, p_values<0.000001)

In [14]:
MASKDIR = '/home/jiunchoi/OFD/source/'
DATADIR = '/home/jiunchoi/OFD/OFD_DATA/derivatives/'
BN_atlas = nib.load(f'{MASKDIR}BNA_3mm_atlas.nii').get_fdata()

def load_brain(group,sub,run):
    task = 'RECALL'
    BN_parcels = np.zeros((246), dtype='object')
    groupsub = '0'+str(group)+'0'+str(sub)
    fmri_data = nib.load(f'{DATADIR}sub-{groupsub}/preprocessed/sub-{groupsub}_{task}_{run}_sc_dt_sm.nii.gz').get_fdata()
    for roi in range (1,246+1):
        roi_data = fmri_data[BN_atlas==roi, :].T #(Time, Voxels)
        roi_data = zscore(roi_data, axis=0)
        roi_data = np.nan_to_num(roi_data)
        BN_parcels[roi-1] = roi_data
    return BN_parcels

In [15]:
groups=[2,3,4,5]
group_brain = np.zeros((len(groups),3), dtype='object')

for group in groups:
    for sub in [1,2,3]:
        sub_brain = []
        if (group==2 and sub==1):
            runs = [1,2,3]
        else:
            runs = [1]
        for run in runs:
            sub_brain.append(load_brain(group,sub,run))
        group_brain[group-2,sub-1] = sub_brain

# ISPC

In [16]:
# 모든 그룹 다 돌아가는 최종판!!
result = np.zeros((4,246,2),dtype='object') #shared_r, paired_r 몰아넣기
result_stat = np.zeros((4,246,2)) # t,p 저장

for group in [2,3,4,5]:
    recalled_evidx_all_oh, recalled_evs_all = extract_recalledidx_all(int(group))
    recalled_evs = extract_recalledidx(int(group))
    wherelist = []
    for i,reao in enumerate(recalled_evidx_all_oh):
        if np.sum(reao)==0 or np.sum(reao)==3:
            a = 'none'
        else:
            a = np.where(reao==1)[0]
        wherelist.append(a)

    for roi in range(246):
        shared_event_patterns = np.zeros((3, len(recalled_evs), group_brain[int(group)-2,int(sub)-1][run-1][roi].shape[1])) #
        paired_event_patterns = np.zeros((3, len(posh[int(group)-2]), group_brain[int(group)-2,int(sub)-1][run-1][roi].shape[1])) #
         
        for sub in [1,2,3]:
            if (group==2 and sub==1):
                runs = [1,2,3]
            else:
                runs = [1]
            eventstartend = []
            runidx = []
            for run in runs:
                evse = get_boundary2(group,sub,run)
                eventstartend.extend(evse)
                runidx.extend([run]*len(evse))

            # Condition 1: 같은사건 패턴로드
            for e in range(len(recalled_evs)):
                ev_from_recall = str(recalled_evs[e][sub-1]).split(',')

                if len(ev_from_recall) > 1:
                        single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                        for efr in ev_from_recall[1:]:
                            single_pattern_timepoint = np.vstack((single_pattern_timepoint,eventstartend[int(float(efr))-1]))
                        single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0,0]/1000):math.ceil(single_pattern_timepoint[0,1]/1000),:]
                        for s in range(1, single_pattern_timepoint.shape[-1]):
                            single_pattern = np.vstack((single_pattern,group_brain[group-2,sub-1][runidx[math.trunc(float(ev_from_recall[0]))-1]-1][roi][math.ceil(single_pattern_timepoint[s,0]/1000):int(single_pattern_timepoint[s,1]/1000),:]))
                else:
                    single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                    single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0]/1000):math.ceil(single_pattern_timepoint[1]/1000),:]
                event_pattern = np.mean(single_pattern,axis=0)
                shared_event_patterns[sub-1,e,:] = event_pattern

            # Condition 2: 다른사건 패턴로드
            for e in range(len(posh[group-2])):
                for protagidx in (wherelist[posh[group-2][e][0]]):
                    if int(float(protagidx)) == (sub-1):
                        ev_from_recall = str(recalled_evs_all[posh[group-2][e][0]][sub-1]).split(',')
                        if len(ev_from_recall) > 1:
                                single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                                for efr in ev_from_recall[1:]:
                                    single_pattern_timepoint = np.vstack((single_pattern_timepoint,eventstartend[int(float(efr))-1]))
                                single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0,0]/1000):math.ceil(single_pattern_timepoint[0,1]/1000),:]
                                for s in range(1, single_pattern_timepoint.shape[-1]):
                                    single_pattern = np.vstack((single_pattern,group_brain[group-2,sub-1][runidx[math.trunc(float(ev_from_recall[0]))-1]-1][roi][math.ceil(single_pattern_timepoint[s,0]/1000):int(single_pattern_timepoint[s,1]/1000),:]))
                        else:
                            single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                            single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0]/1000):math.ceil(single_pattern_timepoint[1]/1000),:]
                        event_pattern = np.mean(single_pattern,axis=0)
                        paired_event_patterns[sub-1,e,:] = event_pattern

        # 공통이벤트 계산
        shared_r = np.zeros(len(recalled_evs))
        for e, ev in enumerate(recalled_evs):
            pattern_r = []
            protagonistidx = wherelist[posh[int(group)-2][e][0]]
            
            if len(protagonistidx)==1:
                pattern_1 = np.squeeze(shared_event_patterns[protagonistidx, e, :])
                for otheridx in [0,1,2]:
                    if otheridx!=protagonistidx:
                        pattern_2 = np.squeeze(shared_event_patterns[otheridx, e, :])
                        pattern_r.append(pearsonr(pattern_1, pattern_2)[0])
                pattern_r = np.mean(pattern_r)
                shared_r[e] = pattern_r

            elif len(protagonistidx)==2:
                otheridx = np.delete(np.array([0,1,2]),wherelist[posh[int(group)-2][e][0]])
                pattern_2 = np.squeeze(shared_event_patterns[otheridx, e, :])
                for protagonistidx_a in protagonistidx:
                    pattern_1 = np.squeeze(shared_event_patterns[protagonistidx_a, e, :])
                    pattern_r.append(pearsonr(pattern_1, pattern_2)[0])
                pattern_r = np.mean(pattern_r)
                shared_r[e] = pattern_r

        #페어이벤트 계산
        pair_r = np.zeros(len(recalled_evs))
        for e, ev in enumerate(recalled_evs):
            pattern_r = []
            protagonistidx = wherelist[posh[int(group)-2][e][0]]
            if len(protagonistidx)==1:
                pattern_1 = np.squeeze(paired_event_patterns[protagonistidx, e, :])
                for otheridx in [0,1,2]:
                    if otheridx!=protagonistidx:
                        pattern_2 = np.squeeze(shared_event_patterns[otheridx, e, :])
                        pattern_r.append(pearsonr(pattern_1, pattern_2)[0])
                pattern_r = np.mean(pattern_r)
                pair_r[e] = pattern_r

            elif len(protagonistidx)==2:
                otheridx = np.delete(np.array([0,1,2]),wherelist[posh[int(group)-2][e][0]])
                pattern_2 = np.squeeze(shared_event_patterns[otheridx, e, :])
                for protagonistidx_a in protagonistidx:
                    pattern_1 = np.squeeze(paired_event_patterns[protagonistidx_a, e, :])
                    pattern_r.append(pearsonr(pattern_1, pattern_2)[0])
                pattern_r = np.mean(pattern_r)
                pair_r[e] = pattern_r

        result[int(group)-2,roi,0] = shared_r        
        result[int(group)-2,roi,1] = pair_r       
        result_stat[int(group)-2,roi,:] = ttest_ind(r2z(shared_r), r2z(pair_r))

In [42]:
np.where(result_paired[:,1]<0.05)[0]

array([  0, 167])

In [30]:
result_paired = np.zeros((246,2))
for roi in range(246):
    sharedr = np.concatenate([result[0,roi,0],result[1,roi,0],result[2,roi,0],result[3,roi,0]])
    pairedr = np.concatenate([result[0,roi,1],result[1,roi,1],result[2,roi,1],result[3,roi,1]])
    result_paired[roi,:] = ttest_ind(r2z(sharedr),r2z(pairedr))

In [39]:
atlas = np.array(nib.load(f'{MASKDIR}BNA_3mm_atlas.nii').get_fdata())

empty_brain = np.zeros((atlas.shape))
for i in range(1,247):
    # if np.mean(result_stat[:,i-1,1],axis=0) < 0.05:
    empty_brain[atlas==i] = result_paired[i-1,0]
empty_brain[empty_brain==0.0] = np.nan
np.save("output/230611 ispc_tmap.npy", empty_brain)

In [51]:
atlas = np.array(nib.load(f'{MASKDIR}BNA_3mm_atlas.nii').get_fdata())

empty_brain = np.zeros((atlas.shape))
for i in range(1,247):
    if result_paired[i-1,1] < 0.05:
        empty_brain[atlas==i] = result_paired[i-1,0]
empty_brain[empty_brain==0.0] = np.nan
np.save("output/230611 ispc_tmap_thres.npy", empty_brain)

In [53]:
min(result_paired[:,0])

-1.662490447157642

In [41]:
corrected_p_paired = np.zeros_like(result_paired[:,1])
corrected_p_paired = multitest.multipletests(result_paired[:,1], method='fdr_bh')[1]
np.where(corrected_p_paired<0.05)[0]

array([], dtype=int64)

# IS-RSA

In [38]:
# 전체 그룹 roi 로드 후 neuralRSM 만들기
results = np.zeros((4,3,246,2)) # group,sub,roi,shared/random
neuralRSMs = np.zeros_like(group_brain, dtype='object')
subj_combinations = list(itertools.combinations(range(3), 2))

for group in [2,3,4,5]:
    lastidx = []
    recalled_evidx_all_oh, recalled_evs_all = extract_recalledidx_all_sharedeb(int(group))
    recalled_evs = extract_recalledidx(int(group))
    
    for roi in range(246): #246
        event_patterns = np.zeros((3, len(recalled_evidx_all_oh), group_brain[int(group)-2,int(sub)-1][run-1][roi].shape[1])) #
         
        for sub in [1,2,3]:
            posthoc_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{group}/sub-0{group}0{sub}_day2_posthoc.xlsx')
            segcol = np.array(posthoc_data['segmentation'])
            lastidx.append(max(segcol))
            
        for sub in [1,2,3]:
            if (group==2 and sub==1):
                runs = [1,2,3]
            else:
                runs = [1]
            eventstartend = []
            runidx = []
            for run in runs:
                evse = get_boundary2(group,sub,run)
                eventstartend.extend(evse)
                runidx.extend([run]*len(evse))

            # 모든 사건 패턴 로드 후 event_patterns[sub-1]에 넣어주기
            for e,ev in enumerate(recalled_evs_all):
                if recalled_evidx_all_oh[e][sub-1] == 1:
                    event_no = str(ev[sub]).split(',') #list of str
                    if len(event_no) > 1:
                        single_pattern_timepoint = eventstartend[int(float(event_no[0]))-1]
                        for efr in event_no[1:]:
                            single_pattern_timepoint = np.vstack((single_pattern_timepoint,eventstartend[int(float(efr))-1]))
                        single_pattern = group_brain[group-2,sub-1][runidx[int(float(event_no[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0,0]/1000):math.ceil(single_pattern_timepoint[0,1]/1000),:]
                        for s in range(1, single_pattern_timepoint.shape[-1]):
                            single_pattern = np.vstack((single_pattern,group_brain[group-2,sub-1][runidx[math.trunc(float(event_no[0]))-1]-1][roi][math.ceil(single_pattern_timepoint[s,0]/1000):int(single_pattern_timepoint[s,1]/1000),:]))
                    else:
                        single_pattern_timepoint = eventstartend[int(float(event_no[0]))-1]
                        single_pattern = group_brain[group-2,sub-1][runidx[int(float(event_no[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0]/1000):math.ceil(single_pattern_timepoint[1]/1000),:]
                    event_pattern = np.mean(single_pattern,axis=0)
                    event_patterns[sub-1,recalled_evs_all[e,0]-1,:] = event_pattern
                        
            # 이제 neuralRSM 만들기 +냅다 다 저장
            neuralRSM = np.zeros((max(recalled_evs_all[:,0]),max(recalled_evs_all[:,0])))
            for i in range(len(neuralRSM)):
                for j in range(i, len(neuralRSM)):
                    if recalled_evidx_all_oh[i,sub-1]==1 and recalled_evidx_all_oh[j,sub-1]==1:
                        neuralRSM[i, j] = pearsonr(event_patterns[sub-1,i,:],event_patterns[sub-1,j,:])[0]
            #print(neuralRSM.shape, 'checkpoint4') #이게 각 사람의 neuralRDM
            neuralRSMs[group-2,sub-1] = neuralRSM

        # 한 그룹 다 저장 후 neuralRSM spearman 계산
        for sc in subj_combinations:
            nRf1 = np.triu(neuralRSMs[group-2,sc[0]]).flatten()
            nRf2 = np.triu(neuralRSMs[group-2,sc[1]]).flatten()
            nRf3 = nRf2[np.random.permutation(nRf2.size)]
            results[group-2,sc[0],roi,0] = spearmanr(nRf1,nRf2)[0]
            results[group-2,sc[0],roi,1] = spearmanr(nRf1,nRf3)[0]
        print(group, roi,'done, checkpoint6')

2 0 done, checkpoint6
2 1 done, checkpoint6
2 2 done, checkpoint6
2 3 done, checkpoint6
2 4 done, checkpoint6
2 5 done, checkpoint6
2 6 done, checkpoint6
2 7 done, checkpoint6
2 8 done, checkpoint6
2 9 done, checkpoint6
2 10 done, checkpoint6
2 11 done, checkpoint6
2 12 done, checkpoint6
2 13 done, checkpoint6
2 14 done, checkpoint6
2 15 done, checkpoint6
2 16 done, checkpoint6
2 17 done, checkpoint6
2 18 done, checkpoint6
2 19 done, checkpoint6
2 20 done, checkpoint6
2 21 done, checkpoint6
2 22 done, checkpoint6
2 23 done, checkpoint6
2 24 done, checkpoint6
2 25 done, checkpoint6
2 26 done, checkpoint6
2 27 done, checkpoint6
2 28 done, checkpoint6
2 29 done, checkpoint6
2 30 done, checkpoint6
2 31 done, checkpoint6
2 32 done, checkpoint6
2 33 done, checkpoint6
2 34 done, checkpoint6
2 35 done, checkpoint6
2 36 done, checkpoint6
2 37 done, checkpoint6
2 38 done, checkpoint6
2 39 done, checkpoint6
2 40 done, checkpoint6
2 41 done, checkpoint6
2 42 done, checkpoint6
2 43 done, checkpoint

In [43]:
result_random = np.zeros((246,2))
for roi in range(246):
    sharedr = np.concatenate([results[0,:,roi,0],results[1,:,roi,0],results[2,:,roi,0],results[3,:,roi,0]])
    randomr = np.concatenate([results[0,:,roi,1],results[1,:,roi,1],results[2,:,roi,1],results[3,:,roi,1]])
    result_random[roi,:] = ttest_ind(r2z(sharedr),r2z(randomr))

(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,) (12,)
(12,

In [44]:
print(np.where(result_random[:,1]<0.05)[0])
corrected_p_random = np.zeros_like(result_random[:,1])
corrected_p_random = multitest.multipletests(result_random[:,1], method='fdr_bh')[1]
print(np.where(corrected_p_random<0.05)[0])

[  0   1   2   3   4   5   6   8   9  10  12  13  14  15  16  17  18  19
  20  21  23  24  25  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  60  61  62  63  64  65  66  67  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100 101 102 103 105 106 107 108 109 110 111 112 114 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 131 132 133 135 136 137
 139 140 141 142 143 145 146 147 148 149 150 151 152 153 154 155 156 158
 159 160 161 162 163 164 165 166 168 169 170 171 172 173 174 175 177 178
 179 181 182 183 184 185 186 187 188 189 190 191 192 193 194 196 197 198
 199 200 201 202 203 204 205 206 207 209 210 211 212 214 215 216 218 219
 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 238 239 240 241 242 243 244 245]
[  0   1   2   3   4   5   6   8   9  10  12  13  14  15  16  17  18  19
  20  21  23  24 

In [48]:
atlas = np.array(nib.load(f'{MASKDIR}BNA_3mm_atlas.nii').get_fdata())

empty_brain = np.zeros((atlas.shape))
for i in range(1,247):
    if corrected_p_random[i-1] < 0.05:
        empty_brain[atlas==i] = result_random[i-1,0]
empty_brain[empty_brain==0.0] = np.nan
np.save("output/230611 isrsa_tmap.npy", empty_brain)

In [50]:
min(result_random[:,0])

1.1495894893029552